# Predicting Rewards with the State Value Function

The state-falue function is a view of the expected return with respect to each state. Below is the equation, where $G$ is the return, $s$ is the state, $\gamma$ is the discount factor, and $r$ is the reward.

$$ V_{\pi}(s) \doteq \mathbb{E}_{\pi}[ G \vert s] = \mathbb{E}_{\pi}\bigg[ \sum^{T}_{k=0} \gamma^k r_{k} \vert s \bigg] $$

You could estimate the expectation in a few ways, but the simplest is to simply average over al of the observed rewards. To investigate how this equation works, you can perform the calculation on a simple environment that is easy to validate.

## The Environment: A Simple Grid World

To demonstrate, let me use a rediculously simple grid-based environment. This consists of 5 squares, with a cliff on the left-hand side and a goal position on the right. Both are terminating states.

In [34]:
#!pip install numpy==1.19.2 &2> /dev/null

In [35]:
from tqdm import tqdm

In [36]:
starting_position = 1 # The starting position
cliff_position = 0 # The cliff position
end_position = 5 # The terminating state position
reward_goal_state = 5 # Reward for reaching goal
reward_cliff = 0 # Reward for falling off cliff

def reward(current_position) -> int:
    if current_position <= cliff_position:
        return reward_cliff
    if current_position >= end_position:
        return reward_goal_state
    return 0

def is_terminating(current_position) -> bool:
    if current_position <= cliff_position:
        return True
    if current_position >= end_position:
        return True
    return False

## The Agent

In this simple environment, let us define an agent with a simple random strategy. On every step, the agent randomly decides to go left or right.

In [37]:
LEFT = -1
RIGHT = 1

In [38]:
def strategy(verbose=False) -> int:
    action = 0
    if np.random.random() >= 0.5:
        action = RIGHT  # Right
    else:
        action = LEFT # Left
    if verbose: 
        if action==LEFT: 
            print('LEFT') 
        else:
            print('RIGHT') 
    return action

## The Experiment

Finally, let's iterate thousands of times and record what happens.

The key to understanding this algorithm is to truly understand that we want to know the return, from a state, on average. Say that out loud. The return, from a state, on average.

You're not rewarding on every step. You're only rewarding when the agent reaches a terminal state. But when you are in the middle of this environment, for example, there is a 50/50 chance of ending up at the goal. You might also end up off the cliff. So in this instance, the expected value of that state is half way between the maximum reward, 5, and the minimum reward, 0.

Note that in this implementation 0 and 5 are terminating states, only 1-4 are valid states, so given four states the mid-point is actually in-between states. This will become clear when you inspect the values later.

If you were to implement this, you need to keep track of which states have been visited and the eventual, final reward. So the implementation below has a simple buffer to keep track of positions.

In [39]:
import numpy as np
verbose = True
np.random.seed(42)

# Global buffers to perform averaging later
value_sum = np.zeros(end_position + 1)
value_sum_list = {0:[], 1:[], 2:[], 3:[], 4:[], 5:[]}
n_hits = np.zeros(end_position + 1)

n_iter = 50
for i in tqdm(range(n_iter)):
    position_history = [] # A log of positions in this episode
    current_position = starting_position # Reset
    while True:
        if verbose: print(f'{current_position}',end=' ')
        # Append position to log
        position_history.append(current_position)

        if is_terminating(current_position):
            if verbose: print('Terminated>')
            break
        
        # Update current position according to strategy
        current_position += strategy(verbose=verbose)

    # Now the episode has finished, what was the reward?
    current_reward = reward(current_position)
    
    # Now add the reward to the buffers that allow you to calculate the average
    for pos in position_history:
        value_sum[pos] += current_reward
        value_sum_list[pos].append(current_reward)
        n_hits[pos] += 1
        
    # Now calculate the average for this episode and print
    expected_return = ', '.join(f'{q:.2f}' for q in value_sum / n_hits)
    print("[{}] Average reward: [{}]".format(i, expected_return))
    print("[{}] n_hits: [{}]".format(i, n_hits))
    print("[{}] list of rewards: [{}]".format(i, value_sum_list))

  0%|          | 0/50 [00:00<?, ?it/s]/var/folders/12/sb6k1cdx1dn2gkn_2l4ty8dr0000gn/T/ipykernel_55781/2619719146.py:36: RuntimeWarning: invalid value encountered in divide
  expected_return = ', '.join(f'{q:.2f}' for q in value_sum / n_hits)
100%|██████████| 50/50 [00:00<00:00, 2611.65it/s]

1 LEFT
0 Terminated>
[0] Average reward: [0.00, 0.00, nan, nan, nan, nan]
[0] n_hits: [[1. 1. 0. 0. 0. 0.]]
[0] list of rewards: [{0: [0], 1: [0], 2: [], 3: [], 4: [], 5: []}]
1 RIGHT
2 RIGHT
3 RIGHT
4 LEFT
3 LEFT
2 LEFT
1 RIGHT
2 RIGHT
3 RIGHT
4 LEFT
3 RIGHT
4 RIGHT
5 Terminated>
[1] Average reward: [0.00, 3.33, 5.00, 5.00, 5.00, 5.00]
[1] n_hits: [[1. 3. 3. 4. 3. 1.]]
[1] list of rewards: [{0: [0], 1: [0, 5, 5], 2: [5, 5, 5], 3: [5, 5, 5, 5], 4: [5, 5, 5], 5: [5]}]
1 LEFT
0 Terminated>
[2] Average reward: [0.00, 2.50, 5.00, 5.00, 5.00, 5.00]
[2] n_hits: [[2. 4. 3. 4. 3. 1.]]
[2] list of rewards: [{0: [0, 0], 1: [0, 5, 5, 0], 2: [5, 5, 5], 3: [5, 5, 5, 5], 4: [5, 5, 5], 5: [5]}]
1 LEFT
0 Terminated>
[3] Average reward: [0.00, 2.00, 5.00, 5.00, 5.00, 5.00]
[3] n_hits: [[3. 5. 3. 4. 3. 1.]]
[3] list of rewards: [{0: [0, 0, 0], 1: [0, 5, 5, 0, 0], 2: [5, 5, 5], 3: [5, 5, 5, 5], 4: [5, 5, 5], 5: [5]}]
1 LEFT
0 Terminated>
[4] Average reward: [0.00, 1.67, 5.00, 5.00, 5.00, 5.00]
[4] n_hits

I've capped the number of episodes to 10 so you can see the evolution of the value estimates. But I encourage you to run this yourself and change this to 10,000.

Note that I have chosen a random seed that stubles right on the second episode. In general you would expect it to reach the goal every 1-in-5. Try changing the seed to see what happens?

You can see that with each episode the value estimate gets closer and closer to the true value (which should be integer 0 to 5). For example, when you are in the state next to the goal (the box next to the end) the you would expect that the agent should stumble towards the goal more often than not. Indeed, 4 out of 5 times it does, which means that the average return is 5 (the goal) multipled by 4/5.

## Discussion

It's worth going through this code line by line. It truly is fundamental. This algorithm allows you to estimate the value of being in each state, purely by experiencing those states.

The key is to remember that the goal is to predict the value FROM each state. The goal is always to reach a point where you are maximizing rewards, so your agent needs to know how far away from optimal it is. This distinction can be tricky to get your head around, but once you have it's hard to think any other way.

You can even use this in your life. Imagine you wanted to achieve some goal. All you have to do is predict the expected return from being in each new state. For example, say you wanted to get into reinforcement learning. You could go back to university, read my book, or go and watch TV. Each of these have value, but with different costs and lengths of time. The expected return of watching TV is probably very low. The expected return of reading my book is high, but doesn't guarantee a job. Going back to uni still doesn't guarantee a job, but it might make it easier to get past HR, but it takes years to achieve. Making decisions in this way is known as using the _expected value_ framework and is useful throughout business and life.